In [1]:
import time
import numpy as np
import json
import networkx as nx
from pprint import pprint
from collections import Counter
import random
import time

In [2]:
with open("stats.txt", mode="r") as f:
    stats = json.load(f)

In [3]:
pprint(stats)

{'average_clustering': [0.005066798238955518, 0.001],
 'average_path_length': [11.748410823170731, 2],
 'degree_cdf': [[0,
                 1,
                 2,
                 3,
                 4,
                 5,
                 6,
                 7,
                 8,
                 9,
                 10,
                 11,
                 12,
                 13,
                 14,
                 15,
                 16,
                 17,
                 19,
                 21,
                 24,
                 46],
                [0.0,
                 0.6902231668437833,
                 0.8517534537725824,
                 0.9086078639744952,
                 0.9378320935175345,
                 0.9516471838469713,
                 0.9654622741764081,
                 0.9723698193411264,
                 0.9776833156216791,
                 0.9808714133900106,
                 0.9845908607863975,
                 0.9888416578108395,
               

In [4]:
def gauss_kernel(r, r_mean, r_sigma):
    gk = (r - r_mean) ** 2
    gk /= 2 * r_sigma ** 2
    gk = np.exp(-gk)
    return gk

In [5]:
def score(G, stats, verbose=False):
    G0 = G.subgraph(
        max(nx.connected_components(G), key=len)
    )

    radius_mean, radius_sigma = stats["radius"]
    diameter_mean, diameter_sigma = stats["diameter"]
    average_clustering_mean, average_clustering_sigma = stats["average_clustering"]
    average_path_length_mean, average_path_sigma = stats["average_path_length"]
    number_cc_mean, number_cc_sigma = stats['number_cc']
    
    r = gauss_kernel(nx.radius(G0), radius_mean, radius_sigma)
    d = gauss_kernel(nx.diameter(G0), diameter_mean, diameter_sigma)
    ac = gauss_kernel(nx.average_clustering(G), average_clustering_mean, average_clustering_sigma)
    aspl = gauss_kernel(nx.average_shortest_path_length(G0), average_path_length_mean, average_path_sigma)
    ncc = gauss_kernel(nx.number_connected_components(G), number_cc_mean, number_cc_sigma)
    
    res = r + d + ac + aspl + ncc 
    
    if verbose:
        print("Radius:", r)
        print("Diameter:", d)
        print("Average clustering:", ac)
        print("Average shortest path length:", aspl)
        print("Number of connected components:", ncc)
        
    return res

In [6]:
def sample_discrete_variable_from_cdf(n, cdf):
    q_seq, p_seq = cdf
    
    intervals = [(q_seq[i] + 1, q_seq[i + 1]) for i in range(len(q_seq) - 1)]
    interval_proba = np.diff(p_seq)
    
    idx_sample = np.random.choice(range(len(intervals)), p=interval_proba, size=n)
    idx_counter = Counter(idx_sample)
    
    sample = []
    for idx, size in idx_counter.items():
        cur_idx_sample = [random.randint(*intervals[idx]) for _ in range(size)]
        sample.extend(cur_idx_sample)
    if sum(sample) % 2 != 0:
        save_idx = np.random.randint(0, n)
        sample[save_idx] += 1
    return np.array(sample)

In [7]:
tries = 2001
best_score = 0
best_graph = None
start_time = time.perf_counter()
    
for i in range(tries):
    G = nx.generators.degree_seq.configuration_model(
        sample_discrete_variable_from_cdf(n=stats["number_nodes"], cdf=stats["degree_cdf"]),
        create_using=nx.Graph
    )
    G.remove_edges_from(nx.selfloop_edges(G))
    
    cur_score = score(G, stats)
    if cur_score > best_score:
        best_graph = G
        best_score = cur_score
    
    if i % 50 == 0:
        print("Total seconds:", time.perf_counter() - start_time)
        print("Iteration:", i)
        print("Score:", best_score)
        print()

Total seconds: 29.2358871
Iteration: 0
Score: 0.12797130792910685

Total seconds: 748.4628898999999
Iteration: 50
Score: 1.006354869391197

Total seconds: 1444.4918054
Iteration: 100
Score: 1.006354869391197

Total seconds: 2147.9659267
Iteration: 150
Score: 1.006354869391197

Total seconds: 2823.6388855
Iteration: 200
Score: 1.006354869391197

Total seconds: 3519.2624227
Iteration: 250
Score: 1.006354869391197

Total seconds: 4220.6527506
Iteration: 300
Score: 1.006354869391197

Total seconds: 4922.4697088
Iteration: 350
Score: 1.006354869391197

Total seconds: 5618.4618674
Iteration: 400
Score: 1.006354869391197

Total seconds: 6294.0381372
Iteration: 450
Score: 1.4665963454218884

Total seconds: 6994.2326159
Iteration: 500
Score: 1.4665963454218884

Total seconds: 7700.6137959
Iteration: 550
Score: 1.4665963454218884

Total seconds: 8486.7604091
Iteration: 600
Score: 1.4665963454218884

Total seconds: 9174.490407099998
Iteration: 650
Score: 1.4665963454218884

Total seconds: 9871.56

In [8]:
G = best_graph.copy()

In [9]:
component_vertices = []
for cc in sorted(nx.connected_components(best_graph), key=len):
    cv = random.choice(list(cc))
    component_vertices.append(cv)

number_cc_mean, _ = stats["number_cc"]
for v in component_vertices:
    if nx.number_connected_components(G) <= number_cc_mean:
        break
    u = random.choice([u for u, _ in G.degree() if u != v and not G.has_edge(u, v)])
    G.add_edge(u, v)

In [10]:
print("Score:", score(G, stats, verbose=True))

Radius: 0.8824969025845955
Diameter: 0.7548396019890073
Average clustering: 4.811470123112115e-05
Average shortest path length: 0.2549912237323524
Number of connected components: 1.0
Score: 2.892375843007186


In [11]:
with open("submission.txt", mode="w") as f:
    res = ""
    for edge in G.edges:
        u, v = edge
        res += f"{u} {v}"
        res += "\n"
    f.write(res)